In [1]:
import pandas as pd
import numpy as np

import scipy
from scipy import spatial

## Notas:
- leer nota de importante
- compatibilidades sin modificar son bastante bajas (las altas están por 0.5. Por eso, se agrega un 0.35 a los resultados)
- para hacer los matches, se usa el Gale-Shapely Algorithm. Vale la pena buscar si hay mejores alternativas. Como se optimiza igual es algo subjetivo. 

In [2]:
 
df = pd.read_csv('PATH')
print(df)

          Marca temporal  \
0    10/02/2022 13:09:02   
1    10/02/2022 13:12:18   
2    10/02/2022 13:12:47   
3    10/02/2022 13:22:17   
4    10/02/2022 13:30:06   
..                   ...   
191   13/02/2022 1:14:42   
192  13/02/2022 10:04:22   
193  13/02/2022 11:10:08   
194  13/02/2022 13:35:56   
195  13/02/2022 15:55:24   

    Estoy de acuerdo con que la información de este quiz sea utilizada única y exclusivamente para los fines establecidos arriba.  \
0                                                   Sí                                                                              
1                                                   Sí                                                                              
2                                                   Sí                                                                              
3                                                   Sí                                                                             

In [3]:
#este sale para facilitar operaciones de los df. 
#lo uso para hacer referencia a las columnas. 
#Si se quisiera hacer referencia a preguntas, sería un nombre muy largo. Por eso está eso 
preguntas = df.columns.values.tolist()



In [4]:
#
#esta celda está para normalizar datos y sacar valores y 
#

#es un df que solo contiene las respuestas de los demás

def get_valores_x_info(df):
    
    valores= df.drop(preguntas[:9], axis =1 )

    old = 5
    new = 2
    normalized_vals=(((valores-1)*new)/old) -1

    #aquí van los nombres, carrera, correo, mail, número, razón, si buscas..., con que te identificas, 
    info_pers = df.drop(preguntas[9:], axis =1 )
    
    return normalized_vals, info_pers

#ejemplo de como usarla
valores, info_pers = get_valores_x_info(df)

In [5]:
#
#funciones que crean una matriz de "compatibilidad"
#

# -una que compara a todos con todos (para los que buscan amigos )
# -otra que pone a una lista contra otra  (para parejas heterosexuales)

#ambas funciones reciben matrices de los puros valores. Se piensan usar en conjunto con 

def compatibilidad_todos (df):
    size = len(df)
    cos_dist_mat = np.zeros((size,size))
    
    for i in range(size):
        for j in range(size):
            person_1 = df.iloc[i].array
            person_2 = df.iloc[j].array
            cos_dist_mat[i][j] = 1-spatial.distance.cosine(person_1,person_2)
            
    return cos_dist_mat

#df1 van a ser los renglones, df2 van a ser las columnas
def compatibilidad_asimet (df1,df2):
    #Al momento de usar esta función, tener en mente quienes ocupan renglones y quienes columnas
    size1 = len(df1)
    size2 = len(df2)
    cos_dist_mat = np.zeros((size1,size2))
    
    for i in range(size1):
        for j in range(size2):
            person_1 = df1.iloc[i].array
            person_2 = df2.iloc[j].array
            cos_dist_mat[i][j] = 1-spatial.distance.cosine(person_1,person_2)
    
    return cos_dist_mat
    


# IMPORTANTE

Por si se desea usar este código en otro test. SE DEBE DE PONER ATENCIÓN a la siguiente celda. Aquí se divide para ver con quien se compara. En esta ocasión se busca minimizar hombres que buscan mujeres (sobran hombres que buscan mujeres) y maximizar mujeres que prefieren hombres. 

In [6]:
#
#calcular distancias coseno de los grupos de interés
#

#falta normalizar a los resultados

# num pregunta dice que buscas de la encuesta:   6
# num pregunta género con el que te identificas: 7
# num pregunta género que buscas:                8

#df de gente que solo buscan amigxs
#  son 52 registros
amigos = df[ df[preguntas[6]] == "Busco solo un amigx"]

#df de mujeres que les gustan los hombres
#  son 42 registros
muj_homb = df[ (df[preguntas[6]] == "Busco al amor de mi vida") & 
             (df[preguntas[7]] == "Mujer") &
             (
              (df[preguntas[8]] == "Hombre") | 
              (df[preguntas[8]] == "Hombre, Mujer, Otro") |
              (df[preguntas[8]] == "Hombre, Mujer")
             )] 

#df de hombres que les gustan las mujeres
#  son 82 registros
homb_muj = df[ (df[preguntas[6]] == "Busco al amor de mi vida") & 
             (df[preguntas[7]] == "Hombre") &
             (df[preguntas[8]] == "Mujer")
             ] 

#df de respuestas de mujeres que les gustan solo mujeres
#  son 3 registros
muj_muj = df[ (df[preguntas[6]] == "Busco al amor de mi vida") & 
             (df[preguntas[7]] == "Mujer") &
             (df[preguntas[8]] == "Mujer")] 

#df de respuestas de hombres que les gustan solo hombres
#  son 15 registros
homb_homb = df[ (df[preguntas[6]] == "Busco al amor de mi vida") & 
             (df[preguntas[7]] == "Hombre") &
             (
                 (df[preguntas[8]] == "Hombre") |
                 (df[preguntas[8]] == "Hombre, Mujer, Otro") |
                 (df[preguntas[8]] == "Hombre, Mujer")
             )] 

# FALTA NORMALIZAR

# dfs que van a usar compatibilidad_todos:
# - amigos
# - muj_muj
# - homb_homb

# dfs que van a usar compatibilidad_asimet
# - homb_muj
# - muj_homb
muj_muj

,Marca temporal,Estoy de acuerdo con que la información de este quiz sea utilizada única y exclusivamente para los fines establecidos arriba.,Nombre completo,Carrera (esta no es parte del test),Correo (para que te podamos mandar los resultados),Número (para que tú y tus matches se puedan contactar) :D,Razón por hacer el quiz,"Si buscas una posible pareja, ¿con que género te identificas?","Si buscas una posible pareja, ¿qué género te atrae? (Puedes seleccionar más de uno)",1. Estoy interesado en el significado de las cosas.,...,"21. Cuando ponen música en una fiesta, yo...",22. Creo en los horóscopos.,"23. Clases, ¿presenciales o en línea?",24. ¿Determinismo o libre albedrío?,25. ¿Eres tóxicx?,26. Le pongo cátsup a la pizza.,27. Indento mi código con...,28. Soy una persona extrovertida.,29. Soy muy crítico de las demás personas.,30. ¿R o Python?
4,10/02/2022 13:30:06,Sí,Carla Andrea Sosa,Ciencia de Datos,csosaper@itam.mx,5534469691,Busco al amor de mi vida,Mujer,Mujer,5,...,4,4,4,5,4,5,5,4,4,5
58,10/02/2022 16:23:13,Sí,Diego Aguilar,Dirección Financiera,diego.aguilarga.98@gmail.con,5518415121,Busco al amor de mi vida,Mujer,Mujer,5,...,4,2,4,4,1,1,5,3,2,1
155,11/02/2022 19:08:38,Sí,Emiliano Rojas,Ingeniería en Negocios,emilianojrojasp@gmail.com,5565762627,Busco al amor de mi vida,Mujer,Mujer,5,...,4,3,2,4,2,1,5,2,3,2


In [7]:
#
# aquí se normaliza y se crean las "matrices de compatibildad"
#

# reminder: las funciones son compatibilidad_todos(df) y compatibilidad_asimet(df1,df2)
# reminder 2: funcion que separa valores de info personal y normaliza es: get_valores_x_info(df)  que regresa (valores, info_pers)
# de aquí en adelante, NO cambiar el órden de los resultados, para no perder referencia. 

#para amigos:
valores_amigos, info_amigos = get_valores_x_info(amigos)
comp_amigos = compatibilidad_todos(valores_amigos)

#para homb_homb
valores_hh, info_hh = get_valores_x_info(homb_homb)
comp_hh = compatibilidad_todos(valores_hh)

#para muj_muj
valores_mm, info_mm = get_valores_x_info(muj_muj)
comp_mm = compatibilidad_todos(valores_mm)

#para homb_muj con muj_homb
#  renglones son mujerer y columnas son hombres
valores_hm, info_hm = get_valores_x_info(homb_muj)
valores_mh, info_mh = get_valores_x_info(muj_homb)
comp_hm_mh = compatibilidad_asimet(valores_mh,valores_hm)
comp_mh_hm = compatibilidad_asimet(valores_hm,valores_mh)

In [8]:
valores_todos, info_todos = get_valores_x_info(df)
comp_todos = compatibilidad_todos(valores_todos)


In [9]:
def nombres_x_comp_1(info,valores):
    renglones = []
    
    #está para crear los nuevos renglones
    nombres = info[preguntas[2]]
    for i in range(len(info)):
        lista = [nombres.array[i]]
        lista_nueva = np.concatenate((lista, valores[i]))

        renglones.append(lista_nueva)
        
    columnas = np.concatenate((["nombres"],info[preguntas[2]].array))
    df = pd.DataFrame(renglones, columns=columnas)
    
    return df

#infoR son los nombres que van a salir como renglones con sus respectivos valores (mujeres). infoC son las columnas (hombres)
def nombres_x_comp_2(infoR,valoresR,infoC):
    renglones = []
    
    #está para crear los nuevos renglones
    nombres = infoR[preguntas[2]]
    for i in range(len(infoR)):
        lista = [nombres.array[i]]
        lista_nueva = np.concatenate((lista, valoresR[i]))

        renglones.append(lista_nueva)
        
    columnas = np.concatenate((["nombres"],infoC[preguntas[2]].array))
    df = pd.DataFrame(renglones, columns=columnas)
    
    return df
#amigos
amigos_final = nombres_x_comp_1(info_amigos, comp_amigos)
#muj_muj
muj_muj_final = nombres_x_comp_1(info_mm, comp_mm)
#homb_homb
homb_homb_final = nombres_x_comp_1(info_hh, comp_hh)
#muj renglones homb columnas
hm_mh_final = nombres_x_comp_2(info_mh, comp_hm_mh,info_hm)
#homb renglones muj columnas
mh_hm_final = nombres_x_comp_2(info_hm, comp_mh_hm,info_mh)

#reemplazar las compatibilidades con uno mismo por -1
homb_homb_final.replace('1.0','-1',inplace=True)
amigos_final.replace('1.0','-1',inplace=True)
muj_muj_final.replace('1.0','-1',inplace=True)
muj_muj_final

,nombres,Carla Andrea Sosa,Diego Aguilar,Emiliano Rojas
0,Carla Andrea Sosa,-1,0.04511788347949419,0.20149595716364266
1,Diego Aguilar,0.04511788347949419,-1,0.4420877689346263
2,Emiliano Rojas,0.20149595716364266,0.4420877689346263,-1


### Matching

Al final va a decir a cada persona, al menos una pareja (nombre y cel), junto con su compatibilidad. En caso de unas mujeres, regresar el nombre 


FALTA
- ver si se puede: cualquier queja (de acoso), que nos reporten a esta cuenta (la de metadata) la persona. 
  - o que "sugerimos que registren una denuncia"

In [10]:
woman_list = info_hm[preguntas[2]]
man_list = info_mh[preguntas[2]]
women_available = {man:woman_list for man in man_list}

#invertí las cosas

#muj  renglones  homb columnas = hm_mh_final = man_df
#homb renglones  muj columnas = mh_hm_final  = woman_df
man_df = mh_hm_final
man_df.index = woman_list
woman_df = hm_mh_final
#woman_df.drop(['nombres'], axis=1,inplace=True)
woman_df.index = man_list
nombres_cols = woman_df.columns.values.tolist()

#pasar a numeric
for nombre in nombres_cols:
    if nombre != nombre:
        woman_df[nombre] = pd.to_numeric(woman_df[nombre])

#woman_df.sort_values(by=['nombres'])
#woman_df.sort_values(by=['nombres'], ascending=True)[woman_df['nombre'] == 'Maria José Velasquez ']
#df.sort_values(by=preguntas[2])[df[preguntas[2]] == 'Tamara Sánchez Ramírez ']#[preguntas[30]]

In [11]:
#### PRUEBAS ######

#              #
#   MATCHING   #
#              #

#una propuesta para un mejor proyecto es usar el siguiente algoritmo. No se ha implementado completamente
#se va a usar el Gale-Shapely Algorithm.
#   regresa parejas de forma que no 
#  https://towardsdatascience.com/gale-shapley-algorithm-simply-explained-caa344e643c2
from copy import copy
from collections import Counter

def gale_shapely(man_list, women_available):
    
    #women available es un diccionario que guarda hombres y su "ranking" con las mujeres
    waiting_list = []
    proposals = {}
    count = 0
    
    # while not all men have pairs
    while len(waiting_list)<len(man_list):
        # man makes proposals
        for man in man_list:
            if man not in waiting_list:
                # each man make proposal to the top women from it's list
                women = women_available[man]
                best_choice = woman_df.loc[man][woman_df.loc[man].index.isin(women)].idxmax()
                proposals[(man, best_choice)]=(woman_df.loc[man][best_choice],
                                                     woman_df.loc[man][best_choice])
        # if women have more than one proposals 
        # she will choose the best option
        overlays = Counter([key[1] for key in proposals.keys()])
        # cycle to choose the best options
        for women in overlays.keys():
            if overlays[women]>1:
                # pairs to drop from proposals
                pairs_to_drop = sorted({pair: proposals[pair] for pair in proposals.keys() 
                        if women in pair}.items(), 
                       key=lambda x: x[1][1]
                      )[1:]
                # if man was rejected by woman
                # there is no pint for him to make proposal 
                # second time to the same woman
                for p_to_drop in pairs_to_drop:
                    del proposals[p_to_drop[0]]
                    _women = copy(women_available[p_to_drop[0][0]])
                    #print("Nombre a dropear ",p_to_drop[0][1])
                    #print("1er elemento: ",_women)
                    for index in _women.index.values:
                        if _women[index] == p_to_drop[0][1]:
                            _women.drop(index)     
                    #_women.remove(p_to_drop[0][1])
                    women_available[p_to_drop[0][0]] = _women
        # man who successfully created pairs must be added to the waiting list 
        waiting_list = [man[0] for man in proposals.keys()]
        #update counter
        print(count)
        count+=1
        
    return proposals

#gale_shapely(man_list,women_available)

In [18]:
#MATCHING idea 2

#sumarle 15% a todos
#se podría pasar a otro for donde estén los df. Para hacer el código menos repetitivo

nom_x_mail = pd.DataFrame()
nom_x_mail[['nombre_mail','mail','numero']] = df[[ preguntas[2], preguntas[4],preguntas[5]]]

nom_x_numero = pd.DataFrame()
nom_x_numero[['nombre_num','numero']] = df[[ preguntas[2],preguntas[5]]]


resultados = pd.DataFrame(columns = ['mail','nombre','match1','comp1', 'match2','comp2', 'match3','comp3'])

#para amigos
for nombre in amigos_final['nombres']:
    amigos_final.sort_values(by=[nombre], ascending=False)
    pareja1 = amigos_final.sort_values(by=[nombre], ascending=False)[['nombres',nombre]].head(3)
    respuestas = pd.DataFrame()
    respuestas['nombre'] = [nombre]
    
    respuestas['match1'] = pareja1['nombres'].head(1).values
    respuestas['comp1'] = pareja1[nombre].head(1).values

    respuestas['match2'] = pareja1['nombres'].head(2).tail(1).values
    respuestas['comp2'] = pareja1[nombre].head(2).tail(1).values

    respuestas['match3'] = pareja1['nombres'].head(3).tail(1).values
    respuestas['comp3'] = pareja1[nombre].head(3).tail(1).values
    
    resultados = pd.concat([resultados,respuestas])
    
#para homb_homb
for nombre in homb_homb_final['nombres']:
    homb_homb_final.sort_values(by=[nombre], ascending=False)
    pareja1 = homb_homb_final.sort_values(by=[nombre], ascending=False)[['nombres',nombre]].head(3)
    respuestas = pd.DataFrame()
    respuestas['nombre'] = [nombre]
    
    respuestas['match1'] = pareja1['nombres'].head(1).values
    respuestas['comp1'] = pareja1[nombre].head(1).values

    respuestas['match2'] = pareja1['nombres'].head(2).tail(1).values
    respuestas['comp2'] = pareja1[nombre].head(2).tail(1).values

    respuestas['match3'] = pareja1['nombres'].head(3).tail(1).values
    respuestas['comp3'] = pareja1[nombre].head(3).tail(1).values
    
    resultados = pd.concat([resultados,respuestas])
    
#para homb_homb
for nombre in muj_muj_final['nombres']:
    muj_muj_final.sort_values(by=[nombre], ascending=False)
    pareja1 = muj_muj_final.sort_values(by=[nombre], ascending=False)[['nombres',nombre]].head(3)
    respuestas = pd.DataFrame()
    respuestas['nombre'] = [nombre]
    
    respuestas['match1'] = pareja1['nombres'].head(1).values
    respuestas['comp1'] = pareja1[nombre].head(1).values

    respuestas['match2'] = pareja1['nombres'].head(2).tail(1).values
    respuestas['comp2'] = pareja1[nombre].head(2).tail(1).values

    respuestas['match3'] = pareja1['nombres'].head(3).tail(1).values
    respuestas['comp3'] = pareja1[nombre].head(3).tail(1).values
    
    resultados = pd.concat([resultados,respuestas])
    

#hay que hacerlo 2 veces, una para mujeres y otra para hombres. 
for nombre in woman_df['nombres']:
    man_df.sort_values(by=[nombre], ascending=False)
    pareja1 = man_df.sort_values(by=[nombre], ascending=False)[['nombres',nombre]].head(3)
    respuestas = pd.DataFrame()
    respuestas['nombre'] = [nombre]
    
    respuestas['match1'] = pareja1['nombres'].head(1).values
    respuestas['comp1'] = pareja1[nombre].head(1).values

    respuestas['match2'] = pareja1['nombres'].head(2).tail(1).values
    respuestas['comp2'] = pareja1[nombre].head(2).tail(1).values

    respuestas['match3'] = pareja1['nombres'].head(3).tail(1).values
    respuestas['comp3'] = pareja1[nombre].head(3).tail(1).values
    
    resultados = pd.concat([resultados,respuestas])

for nombre in man_df['nombres']:
    woman_df.sort_values(by=[nombre], ascending=False)
    pareja1 = woman_df.sort_values(by=[nombre], ascending=False)[['nombres',nombre]].head(3)
    respuestas = pd.DataFrame()
    respuestas['nombre'] = [nombre]
    respuestas['match1'] = pareja1['nombres'].head(1).values
    respuestas['comp1'] = pareja1[nombre].head(1).values

    respuestas['match2'] = pareja1['nombres'].head(2).tail(1).values
    respuestas['comp2'] = pareja1[nombre].head(2).tail(1).values

    respuestas['match3'] = pareja1['nombres'].head(3).tail(1).values
    respuestas['comp3'] = pareja1[nombre].head(3).tail(1).values
    
    resultados = pd.concat([resultados,respuestas])

resultados.sort_values(by=['comp1'], ascending=False)

#52 amigos
#42 muj que buscan hombres
#82 homb que buscan mujeres
#3  muj que buscan muj
#15 homb que buscan homb

#sumarle .16 a todos los resultados
resultados.sort_values(by=['comp1'], ascending=False)
#homb_homb_final[homb_homb_final['nombre'] == 'José Luis Trujillo']
homb_homb_final

,nombres,Eduardo Calvo,Pablo López,José Luis Trujillo,Raul Isaac Resendiz Zayas,Ruben Robles Margueri,Lars Daniel Johansson Niño,Sebastian Ortega Estrada,Mario Aurelio Ventura Pastor,Jose Antonio Villagrana Rebollar,Isaac Gael Soria Cervantes,Pedro Aurelio Sanchez Amador,Armando Peniche Figueroa,Juan Martin Olivares Campos,Pablo Díaz González de la Vega,Álvaro Pérez López
0,Eduardo Calvo,-1,0.16816793501559946,0.2655283184456835,0.1872514715682846,-0.04181920907032355,0.45022390189819883,0.3922591960526507,0.311133276784778,0.15932444118273437,0.05117785010666309,0.0,-0.009014864243364507,0.30050125348238,0.3290546651011026,0.5141111658948059
1,Pablo López,0.16816793501559946,-1,0.6306306306306307,0.2668337169297672,0.45403775653856693,0.2525129539610429,0.20450126683362524,0.6256551962199368,0.29572130029994015,0.30213251254950513,0.2848209673611912,0.2844515571926509,0.31946189452875606,0.5800026463184484,0.4266846113926843
2,José Luis Trujillo,0.2655283184456835,0.6306306306306307,-1,0.5794103567617801,0.17735849864787767,0.3740932651274709,0.2531920446511551,0.5252413992957494,0.1717091421096426,0.31255087505121215,0.2524549483428742,0.1101102802036068,0.4078236951430929,0.5064811841090677,0.4508365705281193
3,Raul Isaac Resendiz Zayas,0.1872514715682846,0.2668337169297672,0.5794103567617801,-1,0.048028345087871016,0.18203016995767474,-0.008240856434303279,0.2941427788752462,0.02421797398482417,0.2027789830667166,0.043823282853182155,0.1009451864143861,0.304854573527817,0.22812937284069357,0.006812817428800799
4,Ruben Robles Margueri,-0.04181920907032355,0.45403775653856693,0.17735849864787767,0.048028345087871016,-1,0.4566112018578744,-0.13803340045540424,0.38928259338372917,-0.045072031960048387,-0.0820416598462419,0.2701656462605566,0.11561164437704419,0.0856399019677927,0.2959132636968116,0.02535864685306055
5,Lars Daniel Johansson Niño,0.45022390189819883,0.2525129539610429,0.3740932651274709,0.18203016995767474,0.4566112018578744,-1,-0.01010926509072263,0.5853502041013761,0.1485439777555766,0.09733854515749141,0.2553558162845788,0.18098521571602488,0.4445336397095926,0.5173022155033641,0.24236616189026794
6,Sebastian Ortega Estrada,0.3922591960526507,0.20450126683362524,0.2531920446511551,-0.008240856434303279,-0.13803340045540424,-0.01010926509072263,-1,0.12523940128139277,0.19591793788175282,-0.05630787729103459,0.08396537142495841,0.24796267748291434,-0.09551338658818387,-0.008830215713766965,0.25236480929567895
7,Mario Aurelio Ventura Pastor,0.311133276784778,0.6256551962199368,0.5252413992957494,0.2941427788752462,0.38928259338372917,0.5853502041013761,0.12523940128139277,-1,0.07360993572178742,0.16971728713381717,0.1331997893779996,0.21241412931374448,0.42541919434278674,0.6513697572856773,0.2830010083507194
8,Jose Antonio Villagrana Rebollar,0.15932444118273437,0.29572130029994015,0.1717091421096426,0.02421797398482417,-0.045072031960048387,0.1485439777555766,0.19591793788175282,0.07360993572178742,-1,0.2978565265446729,-0.20562903028816626,0.3011980859533627,-0.10795837927188279,0.07784989441615231,0.3154109210606648
9,Isaac Gael Soria Cervantes,0.05117785010666309,0.30213251254950513,0.31255087505121215,0.2027789830667166,-0.0820416598462419,0.09733854515749141,-0.05630787729103459,0.16971728713381717,0.2978565265446729,-1,-0.04491516239348048,-0.07427918072759021,0.27511336725792024,0.3495391241135004,0.3072327396526393


In [13]:
#para buscar el número de alguien por nombre

nom_x_numero[ (nom_x_numero['nombre_num'] == 'Daniel Andrés Zúñiga Zamora ') ]
nom_x_numero[ (nom_x_numero['nombre_num'] == 'Santiago') ]


,nombre_num,numero
114,Santiago,5533029027


In [14]:
#falta agregar las otras segmentos de gente (amigos, ... )
# agregarle 0.16 a todos
# dejarlo en enteros

con_mail = pd.merge(nom_x_mail,resultados, left_on='nombre_mail', right_on='nombre',how='right')
con_tel0 = pd.merge(nom_x_numero,con_mail, left_on='nombre_num', right_on='match1',how='right')
con_tel1 = con_tel0.rename(columns={"numero_x": "numero1"})
con_tel2 = con_tel1
con_tel2 = pd.merge(nom_x_numero,con_tel2, left_on='nombre_num', right_on='match2',how='right')
con_tel2 = con_tel2.rename(columns={"numero_x": "numero2",'numero_y':'BORRAR'})
con_tel3 = pd.merge(nom_x_numero,con_tel2, left_on='nombre_num', right_on='match3',how='right')
con_tel3


final = pd.DataFrame
final = con_tel3[['nombre','mail_x','match1','comp1','numero1','match2','comp2','numero_y','match3','comp3','numero_x']]
final = final.rename(columns={'numero_x':'numero3','numero_y':'numero2'})

final['comp1'] = pd.to_numeric(final['comp1'])
final['comp2'] = pd.to_numeric(final['comp2'])
final['comp3'] = pd.to_numeric(final['comp3'])

final['comp1'] = ((final['comp1']+0.19)*100).round()
final['comp2'] = ((final['comp2']+0.19)*100).round()
final['comp3'] = ((final['comp3']+0.19)*100).round()

#final['comp3'][final['comp3'] == -81.0]
final.loc[(final.comp3 == -81.0),'match3']='No había mas gente según tu preferencia'
final.loc[(final.comp3 == -81.0),'numero3']='-'
final.loc[(final.comp3 == -81.0),'comp3']=0


,nombre,mail_x,match1,comp1,numero1,match2,comp2,numero2,match3,comp3,numero3
103,Jimena García,jime23na@gmail.com,Uriel Torices Quiroz,99.0,5541454710,Daniel Andrés Zúñiga Zamora,87.0,5529463674,Emilio Contreras,87.0,5516968406
130,Uriel Torices Quiroz,utoricesq@gmail.com,Jimena García,99.0,5549011616,Isabel de Angoitia y Flores,80.0,5523088679,Andrea Reyes Sánchez,79.0,5544753506
50,Arturo Liera Bravo,alierabr@itam.mx,Salvador Abascal (Chava),98.0,5533204239,Javier David Hernández Estrada,89.0,8332338460,Diana Espinosa Ruiz,85.0,4442859711
36,Salvador Abascal (Chava),sal.abascal09@gmail.com,Arturo Liera Bravo,98.0,4434770218,Alan Vivanco Cordova,95.0,55 81074183,Javier David Hernández Estrada,88.0,8332338460
82,Isabel de Angoitia y Flores,isa.angoitia@gmail.com,Emilio Contreras,97.0,5516968406,Damián Pérez Landeros,86.0,5516947316,Héctor G. T. Torres,85.0,523326374336
...,...,...,...,...,...,...,...,...,...,...,...
62,Isaac Gael Soria Cervantes,bioss2112@gmail.com,Pablo Díaz González de la Vega,54.0,5520935786,José Luis Trujillo,50.0,4492803347,Álvaro Pérez López,50.0,7713829695
61,Jose Antonio Villagrana Rebollar,pepe.917@hotmail.com,Álvaro Pérez López,51.0,7713829695,Armando Peniche Figueroa,49.0,55 7662 3266,Isaac Gael Soria Cervantes,49.0,5578644327
64,Armando Peniche Figueroa,armandopenichef@gmail.com,Jose Antonio Villagrana Rebollar,49.0,5632840530,Pablo López,47.0,5547975986,Pedro Aurelio Sanchez Amador,47.0,5538538409
7,Analiz Frías López,afriasl1@itam.mx,Emilio Tamayo Luisce,49.0,7772349461,Francisco Justo,48.0,5548477529,Abi Ramos Méndez,46.0,5563376857


In [15]:
#final.to_csv('matches.csv')

#final['mail_x'].str.extract(r'(.con)')

#final[final['mail_x'] ]

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
191,.con
192,NaN
193,NaN
194,NaN
